In [2]:
import pandas as pd
import numpy as np
import psycopg2

In [12]:
#establishing the connection
conn = psycopg2.connect(
   database="2022-everything-team12", user='ycm3076', password='C13655814908y!', host='pg.analytics.northwestern.edu', port= '5432'
)
#Creating a cursor object using the cursor() method
cursor = conn.cursor()

#Executing an MYSQL function using the execute() method
cursor.execute("SELECT version();")

# Fetch a single row using fetchone() method.
data = cursor.fetchone()
print("Connection established to: ",data)


Connection established to:  ('PostgreSQL 10.12 on x86_64-pc-linux-gnu, compiled by gcc (GCC) 4.8.5 20150623 (Red Hat 4.8.5-39), 64-bit',)


In [20]:
sks = pd.read_sql("select * from \"SKSTINFO\"", conn);
sks

,SKU,STORE,COST,RETAIL,EXTRA_COL
0,390726,1909,28.0,60.0,0
1,390726,2009,28.0,60.0,0
2,390726,2107,28.0,60.0,0
3,390726,2109,28.0,60.0,0
4,390726,2209,28.0,60.0,0
...,...,...,...,...,...
39230141,390726,1407,28.0,60.0,0
39230142,390726,1507,28.0,60.0,0
39230143,390726,1509,28.0,60.0,0
39230144,390726,1707,28.0,60.0,0


In [55]:
if (conn):
    cursor.close()
    conn.close()
    print("PostgreSQL connection is closed") 

PostgreSQL connection is closed


In [45]:
sks['SKU'].drop_duplicates()

0           390726
36          390730
98          390742
165         390745
507         390748
             ...  
39229801    390696
39229937    390697
39229997    390706
39230050    390714
39230107    390717
Name: SKU, Length: 760212, dtype: int64

In [24]:
sks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39230146 entries, 0 to 39230145
Data columns (total 5 columns):
 #   Column     Dtype  
---  ------     -----  
 0   SKU        int64  
 1   STORE      int64  
 2   COST       float64
 3   RETAIL     float64
 4   EXTRA_COL  int64  
dtypes: float64(2), int64(3)
memory usage: 1.5 GB


In [26]:
sks[['COST','RETAIL','EXTRA_COL']].describe()

,COST,RETAIL,EXTRA_COL
count,3.923015e+07,3.923015e+07,3.923015e+07
mean,2.415759e+01,4.333249e+01,6.229031e-02
std,4.056547e+01,8.392405e+01,2.416821e-01
min,0.000000e+00,0.000000e+00,0.000000e+00
25%,9.000000e+00,1.299000e+01,0.000000e+00
50%,1.536000e+01,2.450000e+01,0.000000e+00
75%,2.676000e+01,4.500000e+01,0.000000e+00
max,2.700000e+03,6.017000e+03,1.000000e+00


Check for SKUs with either cost price or retail price equals 0

In [27]:
sks[(sks['COST']==0) & (sks['RETAIL']==0) ]

,SKU,STORE,COST,RETAIL,EXTRA_COL
7488,392032,304,0.0,0.0,0
7489,392032,1004,0.0,0.0,0
8031,392486,3504,0.0,0.0,0
13787,396822,7603,0.0,0.0,1
16474,397409,404,0.0,0.0,1
...,...,...,...,...,...
39197525,386714,9302,0.0,0.0,0
39197526,386714,9609,0.0,0.0,0
39197527,386714,9909,0.0,0.0,0
39197563,386817,7907,0.0,0.0,0


In [28]:
sks[(sks['COST']==0) & (sks['RETAIL']!=0) ]

,SKU,STORE,COST,RETAIL,EXTRA_COL
75751,409401,102,0.0,0.25,1
75754,409401,202,0.0,0.25,1
75757,409401,302,0.0,0.25,1
75761,409401,402,0.0,0.25,1
75765,409401,502,0.0,0.25,1
...,...,...,...,...,...
38669430,250368,7003,0.0,29.75,0
38717495,262135,1302,0.0,9.75,0
38717496,262135,2502,0.0,9.75,0
38742168,269735,4004,0.0,14.88,0


In [29]:
sks[(sks['COST']!=0) & (sks['RETAIL']==0) ]

,SKU,STORE,COST,RETAIL,EXTRA_COL
398936,496297,107,6.5,0.0,0
398937,496297,209,6.5,0.0,0
398938,496297,304,6.5,0.0,0
398939,496297,307,6.5,0.0,0
398940,496297,403,6.5,0.0,0
...,...,...,...,...,...
38734829,268818,9204,0.5,0.0,0
38734830,268818,9304,0.5,0.0,0
38734831,268818,9404,0.5,0.0,0
38734832,268818,9504,0.5,0.0,0


In [53]:
table_sks = pd.pivot_table(sks,values = ['COST','RETAIL','STORE'],index = 'SKU',
                          aggfunc = {'COST':[min,max,np.mean,'std'],
                                    'RETAIL':[min,max,np.mean,'std'],
                                    'STORE':'count'})
table_sks = table_sks.fillna(0)
table_sks

COST                       RETAIL                                 \
            max    mean     min  std     max        mean     min        std   
SKU                                                                           
3        123.36  123.36  123.36  0.0  440.00  440.000000  440.00   0.000000   
4          9.72    9.72    9.72  0.0   12.00   12.000000   12.00   0.000000   
8         16.56   16.56   16.56  0.0   40.00   40.000000   40.00   0.000000   
15        47.60   47.60   47.60  0.0  119.00  119.000000  119.00   0.000000   
39        47.97   47.97   47.97  0.0  119.00  119.000000  119.00   0.000000   
...         ...     ...     ...  ...     ...         ...     ...        ...   
9999973   33.00   33.00   33.00  0.0   17.00   17.000000   17.00   0.000000   
9999974   18.00   18.00   18.00  0.0   62.25   34.315104   20.50  11.930624   
9999991   51.60   51.60   51.60  0.0  129.00  129.000000  129.00   0.000000   
9999992    4.20    4.20    4.20  0.0   14.00   14.000000   14.00   0.000000   
9999997   15.00   15.00   15.00  0.0   19.50   14.625000    9.75   5.211594   

        STORE  
        count  
SKU            
3         260  
4           1  
8           6  
15        117  
39          3  
...       ...  
9999973     6  
9999974    96  
9999991   103  
9999992   219  
9999997     8  

[760212 rows x 9 columns]